In [3]:
import numpy as np
import pandas as pd
from scipy.stats import halfnorm
from scipy.stats import expon
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib

from sfma import SFMAModel

In [4]:
class Simulator:
    def __init__(
        self,
        nu: int,
        gamma: int,
        sigma_min: float,
        sigma_max: float,
        x: callable,
        func: callable,
        ineff_dist: str = "half-normal",
    ):
        """
        Simulation class for stochastic frontier meta-analysis.

        nu
            The scale of the inefficiency term
        gamma
            The variance of the random effect term
        sigma_min, sigma_max
            The study-specific errors, max and minimum. They will be drawn from a uniform distribution.
        x
            A callable function to generate a realization from a random variable x (is the covariate used
            to construct the frontier). Needs to have an argument size.
        func
            A function of x that defines the frontier
        ineff_dist
            Inefficiency distribution
        """
        self.nu = nu
        self.gamma = gamma
        self.sigma_min = sigma_min
        self.sigma_max = sigma_max
        self.x = x
        self.func = func

        if ineff_dist == "half-normal":
            self.rvs = halfnorm.rvs
        elif ineff_dist == "exponential":
            self.rvs = expon.rvs
        else:
            raise RuntimeError(
                "Inefficiency distribution must be half-normal or exponential"
            )

    def simulate(self, n: int = 1, seed=None, **kwargs):
        if seed is not None:
            np.random.seed(seed)
        sigma = stats.uniform.rvs(
            loc=self.sigma_min, scale=self.sigma_max, size=n
        )
        epsilon = stats.norm.rvs(loc=0, scale=sigma, size=n)

        us = stats.norm.rvs(loc=0, scale=self.gamma, size=n)
        vs = self.rvs(scale=self.nu, size=n)

        xs = self.x(size=n, **kwargs)
        front = self.func(xs)
        observed = front + us - vs + epsilon
        return us, vs, epsilon, sigma, xs, front, observed

In [5]:
np.random.seed(10)

In [6]:
s = Simulator(nu=0.4, # variance of inefficiency
              gamma=0.01, # random effect variance
              sigma_min=0.1, # measurement error lower (will sample uniformly)
              sigma_max=0.2, # measurement error upper (will sample uniformly)
              ineff_dist='half-normal', # inefficiency distribution
              x=lambda size: stats.uniform.rvs(size=size, loc=0.0, scale=10), # sample a single covariate
              func=lambda x: np.log(5 * x + 1.5)) # the functional form of the frontier
us, vs, epsilon, sigma, xs, front, observed = s.simulate(n=1000)

x_front = np.linspace(xs.min(), xs.max())
y_front = s.func(np.linspace(xs.min(), xs.max()))

In [7]:
sim = pd.DataFrame({'output': observed, 'input': xs, 'se': sigma})
sim.sort_values('input', inplace=True)

In [9]:
model = SFMAModel(
    
    # INPUT DATA + COLUMNS
    df=sim, # data frame with below columns in it
    col_output='output', # output column
    col_se='se', # standard error for outputs
    col_input='input', # input column -- only one at this time
    
    # SHAPE CONSTRAINTS
    concave=True, # force concavity of frontier
    convex=False, # force convexity of frontier
    increasing=True, # force increasing trend
    decreasing=False, # force decreasing trend
    r_linear=False, # whether to require linear tails to the right for the spline
    l_linear=False, # whether to require linear tails to the left for the spline
    constr_grid_num=20, # sensible default, if constraints are not being enforced properly, make this larger
    
    # SPLINES
    knots_num=4, # how many knots for spline
    knots_type="domain", # should knots be spaced evenly or based on frequency of data ("frequency")
    knots_degree=3, # degree of the spline, e.g., 3=cubic
    
    # ESTIMATION OF RANDOM EFFECTS
    include_gamma=True, # whether to include random effects -- one per observation
    
    # ESTIMATE AN INTERCEPT
    include_intercept=True, # this should pretty much always be true
    
    # TRIMMING
    pct_trimming=0.01 # what proportion of the data should be trimmed as outliers
)

model.fit()

TypeError: SFMAModel.__init__() got an unexpected keyword argument 'col_output'